# Discounted Cash Flow (DCF) Analysis

This notebook calculates the Enterprise Value and Implied Share Price of a hypothetical company using a DCF model. It starts by projecting revenue and then derives Free Cash Flow (FCF), incorporating EBIT margin expansion. Equity Value is derived from Enterprise Value by subtracting Total Debt and adding Cash & Cash Equivalents. The WACC is calculated within the notebook, including the cost of debt.


In [1]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

import numpy as np
import pandas as pd # Used here for displaying results nicely


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 5.3 MB 3.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 5.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


## 1. Define Base Case Model Inputs

Set the key operational, financial, and WACC assumptions for the DCF model.


In [2]:
# --- Inputs ---

# --- Projection Period ---
projection_years = 5

# --- Revenue ---
initial_revenue = 245.4 # Revenue in Year 0 (Millions)
revenue_growth_rate = 0.08 # Annual revenue growth rate during projection (e.g., 8%)

# --- Profitability & Taxes ---
initial_ebit_margin = 0.1183 # EBIT as a percentage of Revenue in Year 1 (e.g., 15%)
ebit_margin_expansion_per_year = 0 # Annual increase in EBIT margin (e.g., 0.5% or 50bps)
tax_rate = 0.1 # Effective tax rate (e.g., 25%)

# --- Asset & Investment Assumptions (as % of Revenue for simplicity) ---
# Note: More complex models might project these based on historical trends or specific plans.
depreciation_amortization_pct_revenue = 0.03 # D&A as % of Revenue (e.g., 5%)
capex_pct_revenue = 0.08 # Capital Expenditures as % of Revenue (e.g., 7%)
# Change in Net Working Capital (NWC) as % of the *change* in revenue from prior year
nwc_pct_revenue_change = 0 # e.g., 10% of revenue increase goes to NWC

# --- WACC Inputs ---
# Cost of Equity (Ke) Inputs (using CAPM: Ke = Rf + Beta * ERP)
risk_free_rate = 0.03 # Risk-free rate (e.g., long-term government bond yield)
equity_beta = 1.2 # Sectors's beta, see https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/Betas.html
equity_risk_premium = 0.06 # Expected market return premium over risk-free rate

# Cost of Debt (Kd) Inputs - Used to *calculate* pre-tax cost of debt
# Use Book Value of Debt consistent with how Interest Expense is typically reported
total_debt_book = 39 # Total Debt on the balance sheet (book value) (Millions)
interest_expense = 1.4 # Latest annual interest expense (Millions)

# Capital Structure Inputs (Market Values)
# Note: Using book values is simpler but market values are theoretically preferred for weights.
# A more robust model would use current market cap or calculate it.
market_value_equity_estimate = 130.78 # Estimated market capitalization (Millions) - Replace with actual if known
# Market value of debt is often approximated by book value, especially if not publicly traded.
# We use total_debt_book here for consistency with the interest expense calculation.
market_value_debt = total_debt_book # Market value of debt (approximated by book value) (Millions)

# --- Terminal Growth ---
perpetual_growth_rate_base = 0.03 # Perpetual Growth Rate for Terminal Value - Base Case (e.g., 2.5%)

# --- Equity Value Calculation Inputs (Balance Sheet Items) ---
# total_debt_book is already defined above
cash_and_equivalents = 18 # Cash & Cash Equivalents on the balance sheet (Millions)
shares_outstanding = 11.9 # Millions of shares

# Calculate Net Debt for reference (Optional, using book values)
net_debt = total_debt_book - cash_and_equivalents

In [3]:
print("--- Base Case Inputs Defined ---")
print(f"Projection Years: {projection_years}")
print(f"Initial Revenue (Year 0): {initial_revenue:,.2f} M")
print(f"Revenue Growth Rate: {revenue_growth_rate:.2%}")
print(f"Initial EBIT Margin (Year 1): {initial_ebit_margin:.2%}")
print(f"Annual EBIT Margin Expansion: {ebit_margin_expansion_per_year:.3%}")
print(f"Tax Rate: {tax_rate:.2%}")
print(f"D&A (% of Revenue): {depreciation_amortization_pct_revenue:.2%}")
print(f"Capex (% of Revenue): {capex_pct_revenue:.2%}")
print(f"Change in NWC (% of Revenue Change): {nwc_pct_revenue_change:.2%}")
print("\n--- WACC Inputs ---")
print(f"Risk-Free Rate: {risk_free_rate:.2%}")
print(f"Equity Beta: {equity_beta:.2f}")
print(f"Equity Risk Premium: {equity_risk_premium:.2%}")
print(f"Interest Expense: {interest_expense:,.2f} M") # Added Interest Expense
print(f"Total Debt (Book Value): {total_debt_book:,.2f} M") # Used for Kd calculation
print(f"Market Value of Equity (E): {market_value_equity_estimate:,.2f} M")
print(f"Market Value of Debt (D): {market_value_debt:,.2f} M (approximated by book value)")
print("\n--- Terminal Growth & Equity Bridge Inputs ---")
print(f"Base Perpetual Growth Rate: {perpetual_growth_rate_base:.2%}")
# print(f"Total Debt (Book Value): {total_debt_book:,.2f} M") # Already printed above
print(f"Cash & Equivalents: {cash_and_equivalents:,.2f} M")
print(f"(Implied Net Debt: {net_debt:,.2f} M)")
print(f"Shares Outstanding: {shares_outstanding} M")


# %% [markdown]
# ## 2. Calculate Weighted Average Cost of Capital (WACC)
#
# Calculate the WACC using the inputs defined above.
#
# - Cost of Equity (Ke) = Risk-Free Rate + Beta * Equity Risk Premium
# - Pre-tax Cost of Debt (Kd_pretax) = Interest Expense / Total Debt (Book Value)
# - After-tax Cost of Debt (Kd) = Pre-tax Cost of Debt * (1 - Tax Rate)
# - WACC = (E / (E + D)) * Ke + (D / (E + D)) * Kd

# %%
# --- WACC Calculation ---

# Calculate Cost of Equity (Ke) using CAPM
cost_of_equity = risk_free_rate + equity_beta * equity_risk_premium

# Calculate Pre-tax Cost of Debt (Kd_pretax)
if total_debt_book != 0:
    cost_of_debt_pretax = interest_expense / total_debt_book
else:
    cost_of_debt_pretax = 0 # Avoid division by zero if company has no debt
    print("Warning: Total Debt is zero, setting pre-tax cost of debt to 0.")


# Calculate After-tax Cost of Debt (Kd * (1-t))
cost_of_debt_aftertax = cost_of_debt_pretax * (1 - tax_rate)

# Calculate Market Value of Capital (E + D)
# Using market value of debt (approximated by book value here) for weights
market_value_capital = market_value_equity_estimate + market_value_debt

# Calculate Weights (handle case where E+D is zero, though unlikely)
if market_value_capital != 0:
    weight_equity = market_value_equity_estimate / market_value_capital
    weight_debt = market_value_debt / market_value_capital
else:
    weight_equity = 0
    weight_debt = 0
    print("Warning: Market Value of Capital is zero, setting weights to 0.")


# Calculate WACC
wacc_base = (weight_equity * cost_of_equity) + (weight_debt * cost_of_debt_aftertax)

print("\n--- WACC Calculation Results ---")
print(f"Cost of Equity (Ke): {cost_of_equity:.2%}")
print(f"Calculated Pre-tax Cost of Debt: {cost_of_debt_pretax:.2%}") # Show calculated value
print(f"After-Tax Cost of Debt (Kd*(1-t)): {cost_of_debt_aftertax:.2%}")
print(f"Market Value of Capital (E+D): {market_value_capital:,.2f} M")
print(f"Weight of Equity (E / (E+D)): {weight_equity:.2%}")
print(f"Weight of Debt (D / (E+D)): {weight_debt:.2%}")
print(f"-----------------------------------")
print(f"Calculated WACC (Base Case): {wacc_base:.2%}") # This is the base WACC used below


# %% [markdown]
# ## 3. Calculate Projected Free Cash Flows (FCFs) (Base Case)
#
# Project financials year-by-year based on the inputs and calculate Unlevered Free Cash Flow (UFCF).
# UFCF = EBIT * (1 - Tax Rate) + D&A - Capex - Change in NWC
# EBIT Margin increases each year based on the expansion input.

# %%
# --- FCF Projection Calculation (Base Case) ---

# Lists to store projected values
projected_revenue = []
projected_ebit_margin_list = [] # Keep track of the margin each year
projected_ebit = []
projected_nopat = [] # Net Operating Profit After Tax (EBIAT)
projected_depreciation = []
projected_capex = []
projected_change_nwc = []
projected_fcf = []

# Calculate Year 0 NWC for Change in NWC calculation in Year 1
revenue_y0 = initial_revenue
revenue_y_minus_1 = initial_revenue / (1 + revenue_growth_rate)
nwc_y0_change_estimate = (revenue_y0 - revenue_y_minus_1) * nwc_pct_revenue_change

# Project for Year 1 to projection_years
last_revenue = initial_revenue
last_nwc_change = nwc_y0_change_estimate # Use estimate for first year calculation
current_ebit_margin = initial_ebit_margin # Start with the initial margin for Year 1

print("\n--- FCF Projection Details (Base Case) ---")
print(f"{'Year':<6} | {'Revenue':>10} | {'EBIT Mgn':>9} | {'EBIT':>10} | {'NOPAT':>10} | {'D&A':>8} | {'Capex':>8} | {'Chg NWC':>8} | {'FCF':>10}")
print("-" * 95) # Adjusted width for new column

for year in range(1, projection_years + 1):
    # Project Revenue
    current_revenue = last_revenue * (1 + revenue_growth_rate)
    projected_revenue.append(current_revenue)

    # Calculate EBIT using the margin for the *current* year
    # Apply expansion *after* calculating the first year's EBIT
    if year > 1:
         current_ebit_margin += ebit_margin_expansion_per_year # Expand margin from Year 2 onwards

    projected_ebit_margin_list.append(current_ebit_margin) # Store the margin used this year
    current_ebit = current_revenue * current_ebit_margin
    projected_ebit.append(current_ebit)

    # Calculate NOPAT (EBIAT)
    current_nopat = current_ebit * (1 - tax_rate)
    projected_nopat.append(current_nopat)

    # Calculate D&A
    current_depreciation = current_revenue * depreciation_amortization_pct_revenue
    projected_depreciation.append(current_depreciation)

    # Calculate Capex
    current_capex = current_revenue * capex_pct_revenue
    projected_capex.append(current_capex)

    # Calculate Change in NWC
    revenue_increase = current_revenue - last_revenue
    current_change_nwc = revenue_increase * nwc_pct_revenue_change
    projected_change_nwc.append(current_change_nwc)

    # Calculate Free Cash Flow (FCF)
    current_fcf = current_nopat + current_depreciation - current_capex - current_change_nwc
    projected_fcf.append(current_fcf)

    # Print year details
    print(f"{year:<6} | {current_revenue:>10,.2f} | {current_ebit_margin:>9.2%} | {current_ebit:>10,.2f} | {current_nopat:>10,.2f} | {current_depreciation:>8,.2f} | {current_capex:>8,.2f} | {current_change_nwc:>8,.2f} | {current_fcf:>10,.2f}")

    # Update last year's values for next iteration
    last_revenue = current_revenue
    last_nwc_change = current_change_nwc
    # Margin expansion is handled at the start of the loop for the *next* year


# Store base case FCFs for later use
projected_fcf_base = projected_fcf[:] # Create a copy

# %% [markdown]
# ## 4. Calculate Present Value (PV) of Projected FCFs (Base Case)
#
# Discount each year's projected FCF back to its present value using the calculated base WACC.
# PV = FCF / (1 + WACC)^n

# %%
# --- PV of FCF Calculation (Base Case) ---

# Use the calculated wacc_base from Step 2
discount_factors_base = [(1 + wacc_base)**(year) for year in range(1, projection_years + 1)]
pv_fcf_base_list = [fcf / factor for fcf, factor in zip(projected_fcf_base, discount_factors_base)]

# Sum of the present values of the projected FCFs
total_pv_fcf_base = sum(pv_fcf_base_list)

# %% [markdown]
# ## 5. Calculate Terminal Value (TV) (Base Case)
#
# Estimate the value of the company's cash flows beyond the explicit projection period using the base case inputs and calculated WACC.
# TV = (FCF_n * (1 + g)) / (WACC - g)
# where FCF_n is the FCF in the last projection year.

# %%
# --- Terminal Value Calculation (Base Case) ---

# FCF in the last projection year
last_fcf = projected_fcf_base[-1]

# Calculate Terminal Value at the *end* of the projection period
# Ensure perpetual growth is less than WACC
if perpetual_growth_rate_base >= wacc_base:
    raise ValueError("Perpetual growth rate cannot be equal to or exceed WACC.")

terminal_value_base = (last_fcf * (1 + perpetual_growth_rate_base)) / (wacc_base - perpetual_growth_rate_base)

print("\n--- Terminal Value (Base Case) ---")
print(f"(Using WACC = {wacc_base:.2%}, g = {perpetual_growth_rate_base:.2%})")
print(f"Last Projected FCF (Year {projection_years}): {last_fcf:,.2f}")
print(f"Terminal Value (at end of Year {projection_years}): {terminal_value_base:,.2f}")

# %% [markdown]
# ## 6. Calculate Present Value (PV) of Terminal Value (Base Case)
#
# Discount the Terminal Value back to the present day using the calculated base WACC for the length of the projection period.
# PV_TV = TV / (1 + WACC)^n

# %%
# --- PV of Terminal Value Calculation (Base Case) ---

# Discount factor for the terminal value (at the end of the last projection year)
# Uses the last discount factor calculated in Step 4
tv_discount_factor_base = discount_factors_base[-1] # (1 + wacc_base)^projection_years

# Calculate the Present Value of the Terminal Value
pv_terminal_value_base = terminal_value_base / tv_discount_factor_base
enterprise_value_base = total_pv_fcf_base + pv_terminal_value_base

print("--- Present Value of Terminal Value (Base Case) ---")
print(f"(Using WACC = {wacc_base:.2%})")
print(f"Terminal Value Discount Factor (Year {projection_years}): {tv_discount_factor_base:.4f}")
print(f"Present Value of Terminal Value: {pv_terminal_value_base:,.2f}")

# %%
# --- Equity Value and Share Price Calculation (Base Case) ---

# Use book value of debt for the bridge from EV to Equity
if total_debt_book is not None and cash_and_equivalents is not None and shares_outstanding is not None and shares_outstanding != 0:
    # Calculate Equity Value explicitly
    equity_value_base = enterprise_value_base - total_debt_book + cash_and_equivalents
    implied_share_price_base = equity_value_base / shares_outstanding
else:
    implied_share_price_base = np.nan # Set to NaN if cannot be calculated
    equity_value_base = np.nan
    print("\nTotal Debt, Cash, or Shares Outstanding not provided/zero. Skipping Equity Value calculation.")


# %% [markdown]
# ## 10. Sensitivity Analysis: Implied Share Price (WACC vs. Revenue Growth Rate)
#
# Analyze how the Implied Share Price changes based on variations in WACC and the **projection period Revenue Growth Rate**. This requires recalculating FCFs for each scenario, including the **base case EBIT margin expansion**, and uses the base case Book Value Total Debt and Cash. The WACC range is centered around the calculated `wacc_base`.

# %%
# --- Implied Share Price Sensitivity Analysis (WACC vs. Revenue Growth) ---

# Define ranges for sensitivity analysis, centered around calculated base WACC
wacc_range_sp = np.arange(wacc_base - 0.02, wacc_base + 0.021, 0.005) # Adjust range as needed
# Widen the revenue growth range to +/- 5% (0.05)
rev_growth_range_sp = np.arange(revenue_growth_rate - 0.05, revenue_growth_rate + 0.051, 0.01) # Wider range (+/- 5%)

# Function to recalculate the entire DCF and Implied Share Price
# Uses total_debt_book_sens and cash_sens
def calculate_implied_share_price_sens(
    wacc_sens, rev_growth_sens,
    proj_years, init_rev, init_ebit_margin, ebit_margin_exp, tax_rate_sens,
    da_pct_rev, capex_pct_rev, nwc_pct_rev_chg,
    perp_growth_sens, total_debt_book_sens, cash_sens, shares_out_sens
):
    """Recalculates FCFs (with margin expansion), EV, Equity Value, and Implied Share Price for sensitivity."""

    # --- Recalculate FCFs ---
    fcf_list_sens = []
    last_rev_sens = init_rev
    rev_y_minus_1_sens = init_rev / (1 + rev_growth_sens) # Estimate for NWC calc
    nwc_y0_change_sens = (init_rev - rev_y_minus_1_sens) * nwc_pct_rev_chg
    last_nwc_change_sens = nwc_y0_change_sens
    curr_ebit_margin_sens = init_ebit_margin # Start with initial margin for Year 1

    for year in range(1, proj_years + 1):
        curr_rev_sens = last_rev_sens * (1 + rev_growth_sens)

        # Apply margin expansion from Year 2 onwards
        if year > 1:
            curr_ebit_margin_sens += ebit_margin_exp

        curr_ebit_sens = curr_rev_sens * curr_ebit_margin_sens # Use expanding margin
        curr_nopat_sens = curr_ebit_sens * (1 - tax_rate_sens)
        curr_da_sens = curr_rev_sens * da_pct_rev
        curr_capex_sens = curr_rev_sens * capex_pct_rev
        rev_inc_sens = curr_rev_sens - last_rev_sens
        curr_nwc_chg_sens = rev_inc_sens * nwc_pct_rev_chg
        curr_fcf_sens = curr_nopat_sens + curr_da_sens - curr_capex_sens - curr_nwc_chg_sens
        fcf_list_sens.append(curr_fcf_sens)
        last_rev_sens = curr_rev_sens
        last_nwc_change_sens = curr_nwc_chg_sens
        # Margin expansion handled at start of loop for *next* year

    if not fcf_list_sens: # Handle case of 0 projection years if needed
        return np.nan

    # --- Recalculate EV ---
    # Basic check: perpetual growth rate must be less than WACC
    if perp_growth_sens >= wacc_sens:
        return np.nan

    # PV of FCFs
    discount_factors_sp = [(1 + wacc_sens)**y for y in range(1, proj_years + 1)]
    pv_fcf_sp_list = [fcf / factor for fcf, factor in zip(fcf_list_sens, discount_factors_sp)]
    total_pv_fcf_sp = sum(pv_fcf_sp_list)

    # Terminal Value & PV of TV
    last_fcf_sp = fcf_list_sens[-1]
    terminal_value_sp = (last_fcf_sp * (1 + perp_growth_sens)) / (wacc_sens - perp_growth_sens)
    tv_discount_factor_sp = discount_factors_sp[-1] # Discount factor at end of projection
    pv_terminal_value_sp = terminal_value_sp / tv_discount_factor_sp

    # Enterprise Value
    enterprise_value_sp = total_pv_fcf_sp + pv_terminal_value_sp

    # --- Calculate Implied Share Price ---
    if total_debt_book_sens is None or cash_sens is None or shares_out_sens is None or shares_out_sens == 0:
        return np.nan # Cannot calculate if debt/cash/shares missing

    # Calculate Equity Value using Book Value Total Debt and Cash
    equity_value_sp = enterprise_value_sp - total_debt_book_sens + cash_sens
    implied_share_price_sp = equity_value_sp / shares_out_sens

    return implied_share_price_sp

# Create a table to store sensitivity results
sensitivity_results_sp = []

# Iterate through ranges and calculate Implied Share Price
for rg in rev_growth_range_sp:
    row = []
    for w in wacc_range_sp:
        # Call the comprehensive calculation function, passing book total debt and cash
        # Ensure WACC is greater than perpetual growth rate for this specific calculation
        if w > perpetual_growth_rate_base:
            sp = calculate_implied_share_price_sens(
                w, rg,
                projection_years, initial_revenue,
                initial_ebit_margin, ebit_margin_expansion_per_year,
                tax_rate,
                depreciation_amortization_pct_revenue, capex_pct_revenue, nwc_pct_revenue_change,
                perpetual_growth_rate_base, # Use base perpetual growth for this sensitivity
                total_debt_book, cash_and_equivalents, shares_outstanding # Pass book total debt and cash
            )
        else:
            sp = np.nan # Assign NaN if w <= perpetual growth rate
        row.append(sp)
    sensitivity_results_sp.append(row)

# Create a Pandas DataFrame for better display
sensitivity_df_sp = pd.DataFrame(
    sensitivity_results_sp,
    index=[f"{rg:.2%}" for rg in rev_growth_range_sp], # Format rev growth rates for index
    columns=[f"{w:.3%}" for w in wacc_range_sp]      # Format WACC for columns
)

print("\n--- Sensitivity Analysis: Implied Share Price ---")
print(f"(Base Case WACC: {wacc_base:.2%}, Base Case Rev Growth: {revenue_growth_rate:.2%})")
print("\nRows: Revenue Growth Rate (Projection Period)")
print("Columns: WACC")
# Ensure float format is still set
pd.options.display.float_format = '{:.2f}'.format
print(sensitivity_df_sp)

# %% [markdown]
# ## 11. Sensitivity Analysis: Implied Share Price (Revenue Growth Rate vs. Initial EBIT Margin)
#
# Analyze how the Implied Share Price changes based on variations in the **projection period Revenue Growth Rate** and the **Initial EBIT Margin**. This uses the base case WACC and other assumptions.

# %%
# --- Implied Share Price Sensitivity Analysis (Rev Growth vs. EBIT Margin) ---

# Define ranges for sensitivity analysis
# Use the same wide revenue growth range as before
# Define a range for Initial EBIT Margin around the base case
ebit_margin_range_sp = np.arange(initial_ebit_margin - 0.02, initial_ebit_margin + 0.021, 0.005) # e.g., 13.0% to 17.0%

# Create a table to store sensitivity results
sensitivity_results_sp_margin = []

# Iterate through ranges and calculate Implied Share Price
# Rows: Revenue Growth Rate
# Columns: Initial EBIT Margin
for rg in rev_growth_range_sp:
    row = []
    for margin in ebit_margin_range_sp:
        # Call the comprehensive calculation function
        # Use base WACC, vary Rev Growth and Initial EBIT Margin
        # Ensure base WACC is greater than perpetual growth rate (already checked earlier)
        sp = calculate_implied_share_price_sens(
            wacc_base, # Use base WACC
            rg,        # Use Rev Growth from sensitivity range
            projection_years, initial_revenue,
            margin,    # Use Initial EBIT Margin from sensitivity range
            ebit_margin_expansion_per_year, # Keep expansion rate constant
            tax_rate,
            depreciation_amortization_pct_revenue, capex_pct_revenue, nwc_pct_revenue_change,
            perpetual_growth_rate_base,
            total_debt_book, cash_and_equivalents, shares_outstanding
        )
        row.append(sp)
    sensitivity_results_sp_margin.append(row)

# Create a Pandas DataFrame for better display
sensitivity_df_sp_margin = pd.DataFrame(
    sensitivity_results_sp_margin,
    index=[f"{rg:.2%}" for rg in rev_growth_range_sp],     # Format rev growth rates for index
    columns=[f"{m:.2%}" for m in ebit_margin_range_sp]  # Format EBIT margins for columns
)

print("\n--- Sensitivity Analysis: Implied Share Price (Rev Growth vs. Initial EBIT Margin) ---")
print(f"(Base Case Rev Growth: {revenue_growth_rate:.2%}, Base Case Initial EBIT Margin: {initial_ebit_margin:.2%})")
print("\nRows: Revenue Growth Rate (Projection Period)")
print("Columns: Initial EBIT Margin")
# Ensure float format is still set
pd.options.display.float_format = '{:.2f}'.format
print(sensitivity_df_sp_margin)


# %% [markdown]
# ## 11. Summary of Base Case Results

# %%
# --- Display Summary using Pandas DataFrame ---

summary_data = {
    "Metric": [
        "Calculated WACC", # Added WACC
        "Total PV of Projected FCFs",
        "PV of Terminal Value",
        "Enterprise Value (Base Case)",
        "Less: Total Debt (Book Value)", # Updated Label
        "Add: Cash & Equivalents", # Updated Label
        "Equity Value (Base Case)",
        "Shares Outstanding (M)",
        "Implied Share Price (Base Case)"
    ],
    "Value": [
        f"{wacc_base:.2%}", # Added WACC value
        f"{total_pv_fcf_base:,.2f}",
        f"{pv_terminal_value_base:,.2f}",
        f"{enterprise_value_base:,.2f}",
        f"{total_debt_book:,.2f}" if total_debt_book is not None else "N/A", # Show Book Total Debt
        f"{cash_and_equivalents:,.2f}" if cash_and_equivalents is not None else "N/A", # Show Cash
        f"{equity_value_base:,.2f}" if not np.isnan(equity_value_base) else "N/A",
        f"{shares_outstanding}" if shares_outstanding is not None else "N/A",
        f"{implied_share_price_base:.2f}" if not np.isnan(implied_share_price_base) else "N/A"
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n--- DCF Analysis Summary (Base Case) ---")
# Use default pandas print which respects the float format set earlier
print(summary_df.to_string(index=False))

--- Base Case Inputs Defined ---
Projection Years: 5
Initial Revenue (Year 0): 245.40 M
Revenue Growth Rate: 8.00%
Initial EBIT Margin (Year 1): 11.83%
Annual EBIT Margin Expansion: 0.000%
Tax Rate: 10.00%
D&A (% of Revenue): 3.00%
Capex (% of Revenue): 8.00%
Change in NWC (% of Revenue Change): 0.00%

--- WACC Inputs ---
Risk-Free Rate: 3.00%
Equity Beta: 1.20
Equity Risk Premium: 6.00%
Interest Expense: 1.40 M
Total Debt (Book Value): 39.00 M
Market Value of Equity (E): 130.78 M
Market Value of Debt (D): 39.00 M (approximated by book value)

--- Terminal Growth & Equity Bridge Inputs ---
Base Perpetual Growth Rate: 3.00%
Cash & Equivalents: 18.00 M
(Implied Net Debt: 21.00 M)
Shares Outstanding: 11.9 M

--- WACC Calculation Results ---
Cost of Equity (Ke): 10.20%
Calculated Pre-tax Cost of Debt: 3.59%
After-Tax Cost of Debt (Kd*(1-t)): 3.23%
Market Value of Capital (E+D): 169.78 M
Weight of Equity (E / (E+D)): 77.03%
Weight of Debt (D / (E+D)): 22.97%
--------------------------------